In [1]:
import numpy as np

In [2]:
inputarray = np.genfromtxt('example_1.4_0.8.dat')
findchirp_array = np.genfromtxt('example_1.4_0.8_findchirp.dat')

In [3]:
noise_l = np.genfromtxt('noise_spectra/o3_l1.txt') #LIGO Livingston
noise_h = np.genfromtxt('noise_spectra/o3_h1.txt') #LIGO Hanford
noise_v = np.genfromtxt('noise_spectra/o3_v1.txt') #Virgo

noisearray_list = [noise_l,noise_h,noise_v]

In [4]:
import horizondistfuns as hor

In [6]:
quad_dist = hor.horizon_distance_calculation(inputarray,findchirp_array,noisearray_list,'quad')
mean_dist = hor.horizon_distance_calculation(inputarray,findchirp_array,noisearray_list,'mean')

print("The quadrature horizon distance is %.1f Mpc.\n"
     "The mean-of-individual horizon distance is %.1f Mpc." % (quad_dist,mean_dist))

The quadrature horizon distance is 2876.5 Mpc.
The mean-of-individual horizon distance is 1817.4 Mpc.
